 Importing necessary libraries

In [4]:
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import plotly.express as px
import os
import plotly.io as pio

Load Data Function

In [ ]:

def load_data(file_path):
    """
    Load data from a CSV file.

    Parameters:
    file_path (str): The path to the CSV file.

    Returns:
    df (pd.DataFrame): The loaded DataFrame.
    """
    df = pd.read_csv(file_path)
    return df


Data Preparation

In [5]:
def prepare_data(df, fillna_value=None, filters=None, transform_func=None):
    """
    Prepare the DataFrame for visualization.

    Parameters:
    df (pd.DataFrame): The original DataFrame.
    fillna_value: Value to fill missing data. Default is None.
    filters (dict): Dictionary of filters to apply. Example: {'column_name': 'value'}
    transform_func (function): A function to apply transformations to the DataFrame.

    Returns:
    df (pd.DataFrame): The processed DataFrame.
    """
    # Fill missing values if specified
    if fillna_value is not None:
        df = df.fillna(fillna_value)

    # Apply filters if specified
    if filters:
        for column, value in filters.items():
            df = df[df[column] == value]

    # Apply a custom transformation function if specified
    if transform_func:
        df = transform_func(df)

    return df


Color mapping function

In [6]:
def create_color_map(df, category_column):
    """
    Create a color map for unique values in a specified column.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    category_column (str): The column name with categorical data.

    Returns:
    color_discrete_map (dict): A dictionary mapping unique categories to colors.
    """
    # Extract unique categories
    unique_values_list = df[category_column].unique().tolist()
    
    # Define a color map
    color_map = px.colors.sequential.Jet[:len(unique_values_list)]
    color_discrete_map = dict(zip(unique_values_list, color_map))
    
    return color_discrete_map

# Assuming df is your DataFrame and 'category' is the column with unique values
color_discrete_map = create_color_map(df, 'category')

Custom hover text

In [ ]:
# Create custom hover text based on node type
def generate_custom_hover(row, is_child):
    if is_child:
        return (
            f"Platform: "
            f"{row['platform'] if any(part in row['software_manufacturer'] for part in row['platform'].split()) else row['software_manufacturer'] + ' (' + row['platform'] + ')'}<br>"
            f"Tool: "
            f"{row['tool'] if any(part in row['software_manufacturer'] for part in row['tool'].split()) else row['software_manufacturer'] + ' (' + row['tool'] + ')'}<br>"
            f"Primarily used in {row['domain'].lower()}<br>"
            f"Category: {row['category']}<br>"
            f"Proficiency Level: {row['proficiency_level']}<br>"
            f"It is {'an' if row['product_type'] in ['Open Source', 'FaaS'] else 'a'} {row['product_type']} product"
        )
    else:
        return f"{row[column_name]}<br>Proficiency Level: {row['proficiency_level'].sum()}"

# Add custom hover text based on the node type (child or parent)
skills_df['custom_hover'] = skills_df.apply(
    lambda row: generate_custom_hover(row, True),  # Initially set all as children
    axis=1
)


Chart creation

In [7]:
def create_sunburst_chart(df, path, values, color, color_discrete_map, custom_data=None):
    """
    Create a sunburst chart using Plotly Express.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    path (list of str): List of column names to define the hierarchy in the sunburst chart.
    values (str): The column name for values to size the sectors.
    color (str): The column name for colors of the sectors.
    color_discrete_map (dict): A dictionary mapping unique values in the color column to colors.
    custom_data (list of str): List of column names for custom hover data. Default is None.

    Returns:
    fig (go.Figure): The Plotly sunburst figure.
    """
    fig = px.sunburst(
        df,
        path=path,
        values=values,
        color=color,
        color_discrete_map=color_discrete_map,
        custom_data=custom_data
    )
    
    return fig


Display chart function

In [8]:
def display_chart(fig):
    """
    Display a Plotly figure.

    Parameters:
    fig (go.Figure): The Plotly figure to display.
    """
    fig.show()


Main workflow function

In [9]:
def main_workflow(file_path, path_columns, values_column, color_column, color_categories, color_values, custom_hover_data=None):
    """
    Main workflow to load, prepare data, and create a sunburst chart.

    Parameters:
    file_path (str): The path to the CSV file.
    path_columns (list of str): List of column names to define the hierarchy in the sunburst chart.
    values_column (str): The column name for values to size the sectors.
    color_column (str): The column name for colors of the sectors.
    color_categories (list of str): List of category names for the color column.
    color_values (list of str): List of colors corresponding to the category names.
    custom_hover_data (list of str): List of column names for custom hover data. Default is None.
    """
    # Load the data
    df = load_data(file_path)

    # Prepare the data (customize as needed)
    df = prepare_data(df)

    # Create the color map
    color_discrete_map = create_color_map(color_categories, color_values)

    # Create the sunburst chart
    fig = create_sunburst_chart(
        df=df,
        path=path_columns,
        values=values_column,
        color=color_column,
        color_discrete_map=color_discrete_map,
        custom_data=custom_hover_data
    )

    # Display the chart
    display_chart(fig)


Usage:

file_path = 'path_to_your_data.csv'
path_columns = ['domain', 'category', 'software_manufacturer', 'product_type', 'tool']
values_column = 'proficiency_level'
color_column = 'category'
color_categories = ['Category1', 'Category2', 'Category3']
color_values = ['blue', 'green', 'red']
custom_hover_data = ['custom_hover']

main_workflow(file_path, path_columns, values_column, color_column, color_categories, color_values, custom_hover_data)
